In [23]:
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
import pandas as pd


control_stars = [
"TIC 25155310","TIC 307210830","TIC 165159410","TIC 261136679","TIC 32869782",
"TIC 219854185","TIC 279741377","TIC 14884935","TIC 410214986","TIC 99368957",
"TIC 393818343","TIC 2450544","TIC 45822926","TIC 220479565","TIC 168789840",
"TIC 287897364","TIC 16740101","TIC 188350478","TIC 38964113","TIC 377909730",
"TIC 32090583","TIC 296780789","TIC 164512662","TIC 356478651","TIC 271971130",
"TIC 151573176","TIC 402026209","TIC 238176110","TIC 198457221","TIC 42267187",
"TIC 140072337","TIC 310651132","TIC 18763145","TIC 206672575","TIC 36085645",
"TIC 280324392","TIC 118327563","TIC 19283336","TIC 44382762","TIC 259377017",
"TIC 348921203","TIC 28230919","TIC 200164267","TIC 17730455","TIC 294750180",
"TIC 11401737","TIC 33286479","TIC 278920114","TIC 202675839","TIC 14675502"
]


print("Querying NASA Exoplanet Archive for TESS planets...")
planets = NasaExoplanetArchive.query_criteria(
    table="pscomppars",
    select="hostname, disc_facility, discoverymethod",
    where="discoverymethod like 'Transit'"
).to_pandas()

tess_hosts = planets[planets['disc_facility'].str.contains('TESS', na=False)]
planet_targets = tess_hosts['hostname'].unique().tolist()

print(f"Found {len(planet_targets)} TESS planet hosts")
print(f"Control stars: {len(control_stars)}")
print(f"\nFirst 10 planet targets: {planet_targets[:10]}")

Querying NASA Exoplanet Archive for TESS planets...
Found 607 TESS planet hosts
Control stars: 50

First 10 planet targets: ['HD 221416', 'HD 202772 A', 'TOI-1238', 'TOI-1743', 'Ross 176', 'TOI-5422', 'TOI-5799', 'TOI-6223', 'TOI-7510', 'TOI-3856']


In [24]:
import lightkurve as lk
import os
import warnings
warnings.filterwarnings('ignore')

CACHE_DIR = "lc_cache"
os.makedirs(CACHE_DIR, exist_ok=True)

def load_lightcurve(target):
    path = os.path.join(CACHE_DIR, target.replace(" ", "_").replace("-", "_") + ".fits")
    

    if os.path.exists(path):
        try:
            print(f"Loading {target} from cache...")
            lc = lk.read(path)
            return lc.remove_nans().remove_outliers().normalize()
        except Exception as e:
            print(f"Cache read failed for {target}: {e}. Re-downloading...")
            os.remove(path)  
    

    print(f"Searching for {target} in TESS archive...")
    

    search_results = None
    try:
        search_results = lk.search_lightcurve(target, mission='TESS', author='SPOC')
        if len(search_results) == 0:
            print(f"  No SPOC data, trying all authors...")
            search_results = lk.search_lightcurve(target, mission='TESS')
    except Exception as e:
        print(f"  Search error: {e}")
        return None
    
    if search_results is None or len(search_results) == 0:
        print(f"  ✗ No data found for {target}")
        return None
    
    print(f"  Found {len(search_results)} results")
    

    for attempt in range(3):
        try:
            print(f"  Downloading (attempt {attempt + 1}/3)...")
            lc = search_results[0].download()
            
            lc.to_fits(path, overwrite=True)
            print(f"  ✓ Successfully downloaded {target}")
            
            return lc.remove_nans().remove_outliers().normalize()
        except Exception as e:
            print(f"  Download attempt {attempt + 1} failed: {e}")
            if attempt == 2:
                print(f"  ✗ All download attempts failed for {target}")
                return None
    
    return None

In [25]:
from wotan import flatten
import numpy as np

def detrend(lc):
    try:
        time = lc.time.value
        flux = lc.flux.value
        mask = np.isfinite(time) & np.isfinite(flux)
        time = time[mask]
        flux = flux[mask]
        
        if len(time) < 10:
            raise ValueError(f"Insufficient data points: {len(time)}")
        
        flat_flux, _ = flatten(time, flux, method='biweight', window_length=0.5, return_trend=True)
        return time, flat_flux
    except Exception as e:
        print(f"  Detrending error: {e}")
        raise

In [26]:
from transitleastsquares import transitleastsquares
from scipy.stats import median_abs_deviation

def detect_tls(time, flux):
    """Run TLS detection with comprehensive metrics"""
    try:
        model = transitleastsquares(time, flux)
        results = model.power()
        
        if not hasattr(results, 'period') or results.period is None:
            raise ValueError("TLS returned invalid results")
        
        # Return extended results including model fit metrics
        return results
    except Exception as e:
        print(f"  TLS detection error: {e}")
        raise

def calculate_shape_features(time, flux, period, duration, t0):
    """Calculate transit shape features"""
    phase = ((time - t0) % period) / period
    phase[phase > 0.5] -= 1.0  # Center around 0
    
    # Get in-transit points
    in_transit = np.abs(phase) < (duration / period / 2)
    
    if np.sum(in_transit) < 5:
        return 0, 0, 0
    
    transit_flux = flux[in_transit]
    transit_phase = phase[in_transit]
    
    # 1. Transit shape symmetry (compare first half vs second half)
    mid_idx = len(transit_flux) // 2
    first_half = transit_flux[:mid_idx]
    second_half = transit_flux[mid_idx:]
    symmetry = np.std(first_half - second_half[::-1][:len(first_half)]) if len(first_half) > 0 else 0
    
    # 2. Ingress/egress duration ratio (V-shape detection)
    sorted_indices = np.argsort(transit_flux)
    deepest_point = np.median(sorted_indices[:max(1, len(sorted_indices)//5)])
    ingress_points = np.sum(transit_phase < 0)
    egress_points = np.sum(transit_phase > 0)
    shape_ratio = abs(ingress_points - egress_points) / max(ingress_points + egress_points, 1)
    
    # 3. Transit depth variability
    depth_std = np.std(transit_flux)
    
    return symmetry, shape_ratio, depth_std

def odd_even_test_enhanced(time, flux, period, duration, t0):
    """Enhanced odd-even test with multiple metrics"""
    phase = ((time - t0) % period) / period
    phase[phase > 0.5] -= 1.0
    
    # Identify transits
    in_transit = np.abs(phase) < (duration / period / 2)
    transit_number = np.floor((time - t0) / period)
    
    # Separate odd and even transits
    odd_mask = in_transit & (transit_number % 2 == 1)
    even_mask = in_transit & (transit_number % 2 == 0)
    
    if np.sum(odd_mask) < 3 or np.sum(even_mask) < 3:
        return 0, 0, 0
    
    odd_flux = flux[odd_mask]
    even_flux = flux[even_mask]
    
    # 1. Depth difference
    odd_depth = 1 - np.median(odd_flux)
    even_depth = 1 - np.median(even_flux)
    depth_diff = abs(odd_depth - even_depth)
    
    # 2. Duration difference (check if one set is systematically longer)
    odd_duration = np.sum(odd_mask) / len(time) * period
    even_duration = np.sum(even_mask) / len(time) * period
    duration_diff = abs(odd_duration - even_duration) / max(duration, 1e-10)  # Avoid division by zero
    
    # 3. Shape consistency (MAD of transit depths) - Fixed to avoid inf
    mad_odd = median_abs_deviation(odd_flux)
    mad_even = median_abs_deviation(even_flux)
    
    # Safely compute MAD ratio with bounds to prevent inf
    if mad_even > 1e-10 and mad_odd > 1e-10:
        mad_ratio = mad_odd / mad_even
        # Cap the ratio to prevent extreme values
        mad_ratio = np.clip(mad_ratio, 0.01, 100)
        mad_ratio = abs(np.log(mad_ratio))
    else:
        mad_ratio = 0
    
    return depth_diff, duration_diff, mad_ratio

def check_multi_sector(target):
    """Check if target has data from multiple TESS sectors"""
    try:
        search_results = lk.search_lightcurve(target, mission='TESS')
        if len(search_results) > 0:
            # Count unique sectors
            sectors = set()
            for i in range(len(search_results)):
                if hasattr(search_results[i], 'mission') and hasattr(search_results[i], 'sequence_number'):
                    sectors.add(search_results[i].sequence_number)
            return len(sectors)
        return 0
    except:
        return 0


In [27]:
def extract_features(target):
    """Extract comprehensive features from a target star"""
    print(f"\n{'='*60}")
    print(f"Analyzing: {target}")
    print(f"{'='*60}")
    
    def to_scalar(value):
        """Convert numpy array or scalar to Python float, handling inf/nan"""
        if value is None:
            return 0.0
        if isinstance(value, (list, np.ndarray)):
            val = float(value[0]) if len(value) > 0 else 0.0
        else:
            val = float(value)
        
        # Replace inf and nan with 0.0
        if not np.isfinite(val):
            return 0.0
        return val
    
    try:
        # Step 1: Load lightcurve
        lc = load_lightcurve(target)
        if lc is None:
            print(f"✗ Failed to load lightcurve for {target}")
            return None
        
        print(f"  ✓ Lightcurve loaded: {len(lc)} data points")
        
        # Step 2: Multi-sector check
        num_sectors = check_multi_sector(target)
        print(f"  ✓ Available sectors: {num_sectors}")
        
        # Step 3: Detrend
        time, flux = detrend(lc)
        print(f"  ✓ Detrended: {len(time)} clean data points")
        
        # Step 4: TLS detection (get full results object)
        results = detect_tls(time, flux)
        period = to_scalar(results.period)
        duration = to_scalar(results.duration)
        depth = to_scalar(results.depth)
        snr = to_scalar(results.SDE)
        t0 = to_scalar(results.T0)
        
        print(f"  ✓ TLS complete: Period={period:.2f}d, Depth={depth:.4f}, SNR={snr:.2f}")
        
        # Step 5: SDE threshold check
        sde_pass = 1 if snr >= 7.0 else 0  # Standard threshold
        print(f"  ✓ SDE threshold (7.0): {'PASS' if sde_pass else 'FAIL'} (SNR={snr:.2f})")
        
        # Step 6: TLS model fit metrics
        rp_rs = to_scalar(results.rp_rs if hasattr(results, 'rp_rs') else 0)
        snr_pink = to_scalar(results.snr_pink_per_transit if hasattr(results, 'snr_pink_per_transit') else snr)
        odd_even_mismatch = to_scalar(results.odd_even_mismatch if hasattr(results, 'odd_even_mismatch') else 0)
        
        print(f"  ✓ Model fit: Rp/Rs={rp_rs:.4f}, SNR_pink={snr_pink:.2f}")
        
        # Step 7: Shape features
        symmetry, shape_ratio, depth_std = calculate_shape_features(time, flux, period, duration, t0)
        symmetry = to_scalar(symmetry)
        shape_ratio = to_scalar(shape_ratio)
        depth_std = to_scalar(depth_std)
        print(f"  ✓ Shape features: symmetry={symmetry:.6f}, ratio={shape_ratio:.4f}")
        
        # Step 8: Enhanced odd-even test
        depth_diff, duration_diff, mad_ratio = odd_even_test_enhanced(time, flux, period, duration, t0)
        depth_diff = to_scalar(depth_diff)
        duration_diff = to_scalar(duration_diff)
        mad_ratio = to_scalar(mad_ratio)
        print(f"  ✓ Odd-even test: depth_diff={depth_diff:.6f}, dur_diff={duration_diff:.4f}")
        
        # Compile all features (16 features total)
        features = [
            period,           # 0: Orbital period
            depth,            # 1: Transit depth
            duration,         # 2: Transit duration
            snr,              # 3: Signal-to-noise ratio
            sde_pass,         # 4: SDE threshold pass/fail
            rp_rs,            # 5: Planet-to-star radius ratio
            snr_pink,         # 6: SNR accounting for red noise
            odd_even_mismatch,# 7: TLS odd-even mismatch
            symmetry,         # 8: Transit symmetry
            shape_ratio,      # 9: Ingress/egress ratio
            depth_std,        # 10: Transit depth variability
            depth_diff,       # 11: Odd-even depth difference
            duration_diff,    # 12: Odd-even duration difference
            mad_ratio,        # 13: Odd-even MAD ratio
            num_sectors,      # 14: Number of sectors
            len(time)         # 15: Total data points
        ]
        
        # Final sanity check: ensure all features are finite
        for i, feat in enumerate(features):
            if not np.isfinite(feat):
                print(f"  ⚠ Warning: Feature {i} is not finite ({feat}), setting to 0.0")
                features[i] = 0.0
        
        print(f"  ✓ Extracted {len(features)} features successfully")
        return features

    except Exception as e:
        print(f"  ✗ Feature extraction failed: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
        return None


In [28]:

import random
random.seed(42)


planet_sample = random.sample(planet_targets, min(15, len(planet_targets)))

X, y = [], []
failed_targets = []

print("\n" + "="*60)
print(f"COLLECTING PLANET CANDIDATES ({len(planet_sample)} targets)")
print("="*60)

for star in planet_sample:
    feat = extract_features(star)
    if feat:
        X.append(feat)
        y.append(1)
    else:
        failed_targets.append((star, "planet"))

print("\n" + "="*60)
print(f"COLLECTING NON-PLANET STARS ({len(control_stars)} targets)")
print("="*60)

for star in control_stars:
    feat = extract_features(star)
    if feat:
        X.append(feat)
        y.append(0)
    else:
        failed_targets.append((star, "control"))

# Clean data: Remove any samples with inf/nan values
print("\n" + "="*60)
print("CLEANING DATA")
print("="*60)
X_clean, y_clean = [], []
removed_count = 0

for i, (features, label) in enumerate(zip(X, y)):
    # Check if all features are finite
    if all(np.isfinite(f) for f in features):
        X_clean.append(features)
        y_clean.append(label)
    else:
        removed_count += 1
        print(f"  Removed sample {i}: contains inf/nan")

X, y = X_clean, y_clean

print("\n" + "="*60)
print("COLLECTION SUMMARY")
print("="*60)
print(f"✓ Successfully collected: {len(X)} samples")
print(f"  - Planets: {sum(y)}")
print(f"  - Non-planets: {len(y) - sum(y)}")
print(f"  - Removed (inf/nan): {removed_count}")

if failed_targets:
    print(f"\n✗ Failed to process {len(failed_targets)} targets:")
    for target, category in failed_targets:
        print(f"  - {target} ({category})")

if len(X) < 4:
    print("\n⚠ WARNING: Very small dataset! Model may not be reliable.")
else:
    print(f"\n✓ Dataset ready for training!")


COLLECTING PLANET CANDIDATES (15 targets)

Analyzing: TOI-5489
Loading TOI-5489 from cache...
Cache read failed for TOI-5489: The unit 'electron / s' is unrecognized.  It can not be converted to other units.. Re-downloading...
Searching for TOI-5489 in TESS archive...
  Found 5 results
  ✓ Successfully downloaded TOI-5489
  ✓ Lightcurve loaded: 16343 data points
  ✓ Available sectors: 0
  ✓ Detrended: 16343 clean data points
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 37 durations
Searching 16343 data points, 2186 periods from 0.601 to 12.227 days
Using all 8 CPU threads


100%|██████████| 2186/2186 periods | 00:05<00:00


Searching for best T0 for period 3.15076 days


100%|██████████| 6616/6616 [00:00<00:00, 7614.56it/s]


  ✓ TLS complete: Period=3.15d, Depth=0.9987, SNR=10.00
  ✓ SDE threshold (7.0): PASS (SNR=10.00)
  ✓ Model fit: Rp/Rs=0.0326, SNR_pink=1.27
  ✓ Shape features: symmetry=0.003849, ratio=0.0047
  ✓ Odd-even test: depth_diff=0.000231, dur_diff=0.0045
  ✓ Extracted 16 features successfully

Analyzing: TOI-5728
Loading TOI-5728 from cache...
Cache read failed for TOI-5728: The unit 'electron / s' is unrecognized.  It can not be converted to other units.. Re-downloading...
Searching for TOI-5728 in TESS archive...
  Found 34 results
  ✓ Successfully downloaded TOI-5728
  ✓ Lightcurve loaded: 18423 data points
  ✓ Available sectors: 0
  ✓ Detrended: 18423 clean data points
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 38 durations
Searching 18423 data points, 2442 periods from 0.602 to 13.425 days
Using all 8 CPU threads


...

Searching for best T0 for period 4.89141 days


100%|██████████| 12495/12495 [00:02<00:00, 6060.47it/s]


  ✓ TLS complete: Period=4.89d, Depth=0.9980, SNR=20.53
  ✓ SDE threshold (7.0): PASS (SNR=20.53)
  ✓ Model fit: Rp/Rs=0.0403, SNR_pink=13.35
  ✓ Shape features: symmetry=0.000393, ratio=0.3333
  ✓ Odd-even test: depth_diff=0.000020, dur_diff=0.0066
  ✓ Extracted 16 features successfully

Analyzing: TIC 200164267
Searching for TIC 200164267 in TESS archive...
  No SPOC data, trying all authors...
  ✗ No data found for TIC 200164267
✗ Failed to load lightcurve for TIC 200164267

Analyzing: TIC 17730455
Searching for TIC 17730455 in TESS archive...
  No SPOC data, trying all authors...
  ✗ No data found for TIC 17730455
✗ Failed to load lightcurve for TIC 17730455

Analyzing: TIC 294750180
Loading TIC 294750180 from cache...
Cache read failed for TIC 294750180: The unit 'electron / s' is unrecognized.  It can not be converted to other units.. Re-downloading...
Searching for TIC 294750180 in TESS archive...
  Found 46 results
  ✓ Successfully downloaded TIC 294750180
  ✓ Lightcurve loaded

100%|██████████| 2554/2554 periods | 00:06<00:00


Searching for best T0 for period 1.05201 days
  ✓ TLS complete: Period=1.05d, Depth=0.9957, SNR=10.09
  ✓ SDE threshold (7.0): PASS (SNR=10.09)
  ✓ Model fit: Rp/Rs=0.0590, SNR_pink=12.04
  ✓ Shape features: symmetry=0.002084, ratio=0.0097
  ✓ Odd-even test: depth_diff=0.000042, dur_diff=0.0235
  ✓ Extracted 16 features successfully

Analyzing: TIC 11401737
Searching for TIC 11401737 in TESS archive...
  No SPOC data, trying all authors...
  ✗ No data found for TIC 11401737
✗ Failed to load lightcurve for TIC 11401737

Analyzing: TIC 33286479
Searching for TIC 33286479 in TESS archive...
  No SPOC data, trying all authors...
  ✗ No data found for TIC 33286479
✗ Failed to load lightcurve for TIC 33286479

Analyzing: TIC 278920114
Searching for TIC 278920114 in TESS archive...
  No SPOC data, trying all authors...
  ✗ No data found for TIC 278920114
✗ Failed to load lightcurve for TIC 278920114

Analyzing: TIC 202675839
Searching for TIC 202675839 in TESS archive...
  No SPOC data, tryin

In [29]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X, y)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",200
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [ ]:
# Update feature names to match new feature set (16 features)
feature_names = [
    'Period',           # 0: Orbital period (days)
    'Depth',            # 1: Transit depth
    'Duration',         # 2: Transit duration (days)
    'SNR',              # 3: Signal-to-noise ratio (SDE)
    'SDE_Pass',         # 4: SDE threshold pass (1) or fail (0)
    'Rp_Rs',            # 5: Planet-to-star radius ratio
    'SNR_Pink',         # 6: SNR accounting for pink noise
    'OE_Mismatch',      # 7: TLS odd-even mismatch
    'Symmetry',         # 8: Transit symmetry
    'Shape_Ratio',      # 9: Ingress/egress ratio
    'Depth_Std',        # 10: Depth standard deviation
    'OE_Depth_Diff',    # 11: Odd-even depth difference
    'OE_Duration_Diff', # 12: Odd-even duration difference
    'OE_MAD_Ratio',     # 13: Odd-even MAD ratio
    'N_Sectors',        # 14: Number of TESS sectors
    'N_Points'          # 15: Total data points
]


model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X, y)

print(f"✓ Model trained on {len(X)} samples with {len(feature_names)} features")
print(f"  Training accuracy: {model.score(X, y):.2%}")

✓ Model trained on 38 samples with 16 features
  Training accuracy: 100.00%


In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

pred = model.predict(X)
print(classification_report(y, pred))

# ConfusionMatrixDisplay.from_estimator(model, X, y)
# plt.show()


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        15

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



In [33]:
def predict_star(target, save_to_csv=True):
    """Predict whether a target has a planet and optionally save to CSV"""
    from pathlib import Path
    
    print(f"\n{'='*70}")
    print(f"PREDICTION FOR: {target}")
    print(f"{'='*70}")
    
    # Extract features
    features = extract_features(target)
    
    if features is None:
        print(f"\n✗ Could not make prediction for {target} (feature extraction failed)")
        return None
    
    # Make prediction
    features_array = np.array(features).reshape(1, -1)
    prediction = model.predict(features_array)[0]
    probability = model.predict_proba(features_array)[0][1]  # Probability of being a planet
    
    result = "Planet" if prediction == 1 else "Non-Planet"
    
    print(f"\n{'='*70}")
    print(f"PREDICTION RESULT: {result}")
    print(f"Planet Probability: {probability:.2%}")
    print(f"{'='*70}")
    
    # Save to CSV if requested
    if save_to_csv:
        import pandas as pd
        from datetime import datetime
        
        csv_file = 'predictions.csv'
        
        # Create prediction record
        record = {
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'target_name': target,
            'prediction': result,
            'planet_probability': probability
        }
        
        # Add all features
        for i, name in enumerate(feature_names):
            record[name.lower().replace(' ', '_')] = features[i]
        
        # Append to CSV
        df_new = pd.DataFrame([record])
        
        try:
            # Read existing file if it exists
            if Path(csv_file).exists():
                df_existing = pd.read_csv(csv_file)
                df_combined = pd.concat([df_existing, df_new], ignore_index=True)
            else:
                df_combined = df_new
            
            # Save combined data
            df_combined.to_csv(csv_file, index=False)
            print(f"✓ Prediction saved to {csv_file}")
        except Exception as e:
            print(f"✗ Failed to save to CSV: {e}")
    
    return {'target': target, 'prediction': result, 'probability': probability, 'features': features}

In [ ]:
print("Testing the model on new targets:")
print("="*60)

predict_star("TOI-270")

print()

predict_star("TIC 38846515")

Testing the model on new targets:

Analyzing: TOI-270
Loading TOI-270 from cache...
Cache read failed for TOI-270: The unit 'electron / s' is unrecognized.  It can not be converted to other units.. Re-downloading...
Searching for TOI-270 in TESS archive...


  Found 9 results
  ✓ Successfully downloaded TOI-270
  ✓ Lightcurve loaded: 12982 data points
  ✓ Detrended: 12982 clean data points
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 36 durations
Searching 12982 data points, 1745 periods from 0.602 to 10.139 days
Using all 8 CPU threads


100%|██████████| 1745/1745 periods | 00:05<00:00


Searching for best T0 for period 5.66230 days


100%|██████████| 12982/12982 [00:01<00:00, 10075.02it/s]


  ✓ TLS complete: Period=5.66d, Depth=0.9968, SNR=13.69
  ✓ Odd-even test: depth_diff=0.000016
  ✓ Features extracted successfully
✓ Saved to predictions.csv
⭐ Likely NOT a planet. Probability of planet: 0.16


Analyzing: TIC 38846515
Loading TIC 38846515 from cache...
Cache read failed for TIC 38846515: The unit 'electron / s' is unrecognized.  It can not be converted to other units.. Re-downloading...
Searching for TIC 38846515 in TESS archive...
  Found 64 results
  ✓ Successfully downloaded TIC 38846515
  ✓ Lightcurve loaded: 15871 data points
  ✓ Detrended: 15871 clean data points
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 38 durations
Searching 15871 data points, 2362 periods from 0.601 to 13.051 days
Using all 8 CPU threads


100%|██████████| 2362/2362 periods | 00:07<00:00


Searching for best T0 for period 2.84876 days
  ✓ TLS complete: Period=2.85d, Depth=0.9926, SNR=20.13
  ✓ Odd-even test: depth_diff=0.000080
  ✓ Features extracted successfully
✓ Saved to predictions.csv
⭐ Likely NOT a planet. Probability of planet: 0.38


{'timestamp': '2026-02-05 11:39:18',
 'target_name': 'TIC 38846515',
 'prediction': 'Non-Planet',
 'planet_probability': np.float64(0.375),
 'period_days': np.float64(2.8488),
 'depth': np.float64(0.992552),
 'duration_days': np.float64(0.12267),
 'snr': np.float64(20.1298),
 'odd_even_diff': np.float64(8e-05)}

In [31]:
test_features = extract_features("TIC 307210830")
if test_features:
    print(f"\n\nExtracted {len(test_features)} features:")
    print(f"  Period: {test_features[0]:.2f} days")
    print(f"  Depth: {test_features[1]:.6f}")
    print(f"  Duration: {test_features[2]:.4f} days")
    print(f"  SNR: {test_features[3]:.2f}")
    print(f"  SDE Pass: {test_features[4]}")


Analyzing: TIC 307210830
Loading TIC 307210830 from cache...
Cache read failed for TIC 307210830: The unit 'electron / s' is unrecognized.  It can not be converted to other units.. Re-downloading...
Searching for TIC 307210830 in TESS archive...
  Found 45 results
  ✓ Successfully downloaded TIC 307210830
  ✓ Lightcurve loaded: 17577 data points
  ✓ Available sectors: 0
  ✓ Detrended: 17577 clean data points
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 38 durations
Searching 17577 data points, 2390 periods from 0.601 to 13.182 days
Using all 8 CPU threads


100%|██████████| 2390/2390 periods | 00:07<00:00


Searching for best T0 for period 3.69144 days


100%|██████████| 7642/7642 [00:01<00:00, 6057.69it/s]

  ✓ TLS complete: Period=3.69d, Depth=0.9983, SNR=21.04
  ✓ SDE threshold (7.0): PASS (SNR=21.04)
  ✓ Model fit: Rp/Rs=0.0370, SNR_pink=10.71
  ✓ Shape features: symmetry=0.001230, ratio=0.0208
  ✓ Odd-even test: depth_diff=0.000092, dur_diff=0.0184
  ✓ Extracted 16 features successfully


Extracted 16 features:
  Period: 3.69 days
  Depth: 0.998312
  Duration: 0.0457 days
  SNR: 21.04
  SDE Pass: 1


In [34]:
test_targets = ["TIC 16740101","TIC 188350478","TIC 38964113","TIC 377909730"]

print("Running batch predictions...")
print("="*60)

for target in test_targets:
    predict_star(target)
    print()

print("\n" + "="*60)
print("PREDICTIONS SUMMARY")
print("="*60)
print(f"\n✓ All predictions saved to: predictions.csv")

Running batch predictions...

PREDICTION FOR: TIC 16740101

Analyzing: TIC 16740101
Loading TIC 16740101 from cache...
Cache read failed for TIC 16740101: The unit 'electron / s' is unrecognized.  It can not be converted to other units.. Re-downloading...
Searching for TIC 16740101 in TESS archive...
  Found 8 results
  ✓ Successfully downloaded TIC 16740101
  ✓ Lightcurve loaded: 13816 data points
  ✓ Available sectors: 0
  ✓ Detrended: 13816 clean data points
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 38 durations
Searching 13816 data points, 2443 periods from 0.601 to 13.425 days
Using all 8 CPU threads


100%|██████████| 2443/2443 periods | 00:06<00:00


Searching for best T0 for period 1.48108 days
  ✓ TLS complete: Period=1.48d, Depth=0.9929, SNR=34.25
  ✓ SDE threshold (7.0): PASS (SNR=34.25)
  ✓ Model fit: Rp/Rs=0.0759, SNR_pink=110.24
  ✓ Shape features: symmetry=0.000865, ratio=0.0103
  ✓ Odd-even test: depth_diff=0.000015, dur_diff=0.0222
  ✓ Extracted 16 features successfully

PREDICTION RESULT: Non-Planet
Planet Probability: 13.00%
✓ Prediction saved to predictions.csv


PREDICTION FOR: TIC 188350478

Analyzing: TIC 188350478
Searching for TIC 188350478 in TESS archive...
  No SPOC data, trying all authors...
  ✗ No data found for TIC 188350478
✗ Failed to load lightcurve for TIC 188350478

✗ Could not make prediction for TIC 188350478 (feature extraction failed)


PREDICTION FOR: TIC 38964113

Analyzing: TIC 38964113
Searching for TIC 38964113 in TESS archive...
  No SPOC data, trying all authors...
  ✗ No data found for TIC 38964113
✗ Failed to load lightcurve for TIC 38964113

✗ Could not make prediction for TIC 38964113 (f

100%|██████████| 2588/2588 periods | 00:07<00:00


Searching for best T0 for period 7.82947 days


100%|██████████| 14208/14208 [00:01<00:00, 8296.23it/s]


  ✓ TLS complete: Period=7.83d, Depth=0.9827, SNR=6.27
  ✓ SDE threshold (7.0): FAIL (SNR=6.27)
  ✓ Model fit: Rp/Rs=0.1184, SNR_pink=3.20
  ✓ Shape features: symmetry=0.044029, ratio=0.0216
  ✓ Odd-even test: depth_diff=0.006305, dur_diff=0.0083
  ✓ Extracted 16 features successfully

PREDICTION RESULT: Non-Planet
Planet Probability: 16.00%
✓ Prediction saved to predictions.csv


PREDICTIONS SUMMARY

✓ All predictions saved to: predictions.csv


In [ ]:
# Generate visualizations from saved predictions

# visualize_predictions('predictions.csv')

In [ ]:
predict_star("Ross 176")

Adding a new prediction to test append functionality...

PREDICTION FOR: Ross 176

Analyzing: Ross 176
Loading Ross 176 from cache...
Cache read failed for Ross 176: The unit 'electron / s' is unrecognized.  It can not be converted to other units.. Re-downloading...
Searching for Ross 176 in TESS archive...
  Found 5 results
  ✓ Successfully downloaded Ross 176
  ✓ Lightcurve loaded: 19612 data points
